# Лабораторная работа #3
### Варламов Никита, М33361; Гайнанов Ильдар, М33371;

# Task 1

Предъявите доверительный интервал уровня $1 - \alpha$ для указанного параметра при данных предположениях (с математическими обоснованиями). Сгенерируйте 2 выборки объёма 25 и посчитайте доверительный интервал. Повторить 1000 раз. Посчитайте, сколько раз 95-процентный доверительный интервал покрывает реальное значение параметра. То же самое сделайте для объема выборки 10000. Как изменился результат? Как объяснить?
Задача представлена в 4 вариантах. Везде даны две независимые выборки $X, Y$ из нормальных распределений $\mathcal{N} (\mu_1, \sigma_1^2), \mathcal{N} (\mu_2, \sigma_2^2)$ объема $n, m$ соответственно. Сначала указывается оцниваемая функция, потом данные об остальных параметрах, затем параметры эксперимента и подсказки.

Оцениваем $\tau = \mu_1 - \mu_2$. Известны $\sigma_1^2, \sigma_2^2$.
Для эксперимента берём $\mu_1 = 2, \mu_2 = 1$

Построим доверительный интервал по определению для функции статистики $\frac{\overline{X} - \overline{Y} - \tau}{\sigma}$. Очевидно, что она распределена по $\mathcal{N} (0, 1)$ ($\overline{X} \sim \mathcal{N} \left(\mu_1, \frac{\sigma_1^2}{n}\right)$, $\overline{Y} \sim \mathcal{N} \left(\mu_2, \frac{\sigma_2^2}{n}\right)$, $\overline{X} - \overline{Y} \sim \mathcal{N} \left(\mu_1 - \mu_2, \frac{\sigma_1^2}{n} + \frac{\sigma_2^2}{n}\right), \ldots$

По определению (квантили одинаковые, тк у нас нормальное распределение):

$P\left(-q_{1-\frac{\alpha}{2}} \le \frac{\overline{X} - \overline{Y} - \tau}{\sigma} \le q_{1-\frac{\alpha}{2}}\right) = 1 - \alpha$

Выражаем $\tau$:

$P\left(\tau \in \left[\overline{X} - \overline{Y} -q_{1-\frac{\alpha}{2}} \sigma, \overline{X} - \overline{Y} +q_{1-\frac{\alpha}{2}} \sigma\right] \right)$

Проводим эксперимент:

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

ns = [25, 10000]

tau1 = 2
tau2 = 1

sigma1 = 1
sigma2 = 0.5

def experiment(n, count=1000):
    dataX = np.random.normal(loc=tau1, scale=sigma1, size=(count, n))
    dataY = np.random.normal(loc=tau2, scale=sigma2, size=(count, n))
    
    meanX = dataX.mean(axis=1)
    meanY = dataY.mean(axis=1)
    
    quantile = norm.ppf(0.95, loc=0, scale=1)
    sigma = np.sqrt(sigma1 / n + sigma2 / n)
    
    df = pd.DataFrame()
    df['lower_bound'] = meanX - meanY - quantile * sigma
    df['upper_bound'] = meanX - meanY + quantile * sigma
    df['includes'] = ((df['lower_bound'] <= 1) & (df['upper_bound'] >= 1)).astype(bool)
 
    return df

In [2]:
e25 = experiment(25)
e25

,lower_bound,upper_bound,includes
0,0.828144,1.633954,True
1,0.289975,1.095785,True
2,0.654391,1.460201,True
3,0.687897,1.493708,True
4,0.759017,1.564827,True
...,...,...,...
995,0.396545,1.202356,True
996,0.273035,1.078845,True
997,0.530575,1.336386,True
998,0.470167,1.275978,True


In [3]:
"1 in interval", len(e25[e25['includes'] == True])

('1 in interval', 921)

In [4]:
e10000 = experiment(10000)
e10000

,lower_bound,upper_bound,includes
0,0.982176,1.022467,True
1,0.986712,1.027002,True
2,0.967795,1.008086,True
3,0.978156,1.018447,True
4,0.979272,1.019563,True
...,...,...,...
995,0.988458,1.028749,True
996,0.979867,1.020157,True
997,0.988241,1.028532,True
998,0.958260,0.998551,False


In [5]:
"1 in interval", len(e10000[e10000['includes'] == True])

('1 in interval', 939)

Как можно видеть, оценка достаточно точная в обоих экспериментах. Однако, сильно отличется длина интервала

In [6]:
(e25['upper_bound'] - e25['lower_bound']).mean()

0.8058104175194676

In [7]:
(e10000['upper_bound'] - e10000['lower_bound']).mean()

0.0402905208759733

# Task 2



$$ Geom(p): $$

$$ P(X = x) = (1 - p)^{x - 1}\cdot p$$

$$ f_p = \prod P_i = \prod (1 - p)^{\sum(x_i - 1)}\cdot p^{n}$$

$$ L(p) = \sum ((x_i - 1)ln(1 - p) + ln(p))$$

$$ L'(p) = \dfrac{\sum(1-x_i)}{1-p} + \dfrac{n}{p}$$

$$L  \rightarrow max\  \text{в} \  p = \dfrac{n}{\sum x_i} = \dfrac{1}{\overline{x}}$$

$$\overline{p} = \dfrac{1}{\overline{x}}$$

$$P\left(\tau \in \left[\overline{p} -q_{1-\frac{\alpha}{2}} \sigma, \overline{p} +q_{1-\frac{\alpha}{2}} \sigma\right] \right)$$

In [8]:
import numpy as np
import pandas as pd
from scipy.stats import rv_continuous, geom

ns = [25, 10000]

def experiment(n, count=1000, alpha=0.05):
    data = np.random.normal(0.7, size=(count, n))
    mean = data.mean(axis=1)
    
    quantile = geom.ppf(1 - alpha / 2, p=0.7)
    sigma = np.sqrt(sigma1 / n + sigma2 / n)
    
    df = pd.DataFrame()
    df['lower_bound'] = mean - quantile * sigma
    df['upper_bound'] = mean + quantile * sigma
    df['includes'] = ((df['lower_bound'] <= 0.7) & (df['upper_bound'] >= 0.7)).astype(bool)
 
    return df

In [9]:
"1 in interval", len(e25[e25['includes'] == True])

('1 in interval', 921)

In [10]:
e10000 = experiment(10000)
e10000

,lower_bound,upper_bound,includes
0,0.632972,0.730951,True
1,0.660824,0.758804,True
2,0.634472,0.732451,True
3,0.661428,0.759408,True
4,0.657643,0.755623,True
...,...,...,...
995,0.639436,0.737415,True
996,0.663615,0.761594,True
997,0.633876,0.731856,True
998,0.621817,0.719797,True


In [11]:
"1 in interval", len(e10000[e10000['includes'] == True])

('1 in interval', 1000)

In [12]:
(e25['upper_bound'] - e25['lower_bound']).mean()

0.8058104175194676

In [13]:
(e10000['upper_bound'] - e10000['lower_bound']).mean()

0.09797958971132714